  #                                                  Projet Scraping :
## presentateur :  

Papa Samba Dia   
Mor Diouf



# Objectif Du Projet:
  Ce projet intitule **Jumia_Bot** a pour objectif de scraper le site commerciale (https://www.jumia.sn) pour recuperer tous les informations en rapport avec les produits (Telephones et Tablettes) vendus sur jumia.

  Jumia est une entreprise de commerce en ligne présente sur le marché africain et fondée en 2012. La plateforme de Jumia est un marché en ligne qui met en relation des vendeurs et des acheteurs, en mettant à leur disposition un service logistique, permettant l'expédition et la livraison des colis en plus d'un service de paiement.

### ***Information cible:***  
Titre et caracteristique   
marque  
Prix   
Image  
Lien du Produit  
Prix de livraison  
Note  
Lieu de Livraison.

## Requirements
**beautifulsoup4** - 
 **requests** -
**pandas**


In [1]:
import requests
from bs4 import BeautifulSoup  

#La fonction get_product_title Title qui retourne le titre du produit (telephone ou Tablette)
def get_product_title(soup):
    return soup.find('h1',{'class':'-fs20 -pts -pbxs'}).get_text()


#La fonction get_product_price qui retourne le prix du produit (telephone ou Tablette)
def get_product_price(soup):
    soup=soup.find('span',{'class':'-b -ltr -tal -fs24'}).get_text()
    return soup[:-1]


#La fonction get_product_mark qui retourne la marque du produit (telephone ou Tablette)
def get_product_mark(soup):
    s=soup.find('div',{'class':'-phs'})
    return soup.findAll('div',{'class':'-pvxs'})


#La fonction get_price_delivery qui retourne le prix de livraison du produit (telephone ou Tablette)
def get_price_delivery(soup):
    return soup.find('div',{'class':'markup -fs12 -pbm'}).get_text()


#La fonction get_numbers_star qui retourne la note recus par le produit (telephone ou Tablette)
def get_numbers_star(soup):
    return soup.find('div',{'class':'stars _s _al'}).get_text()


#La fonction get_delivery_to qui retourne le lieu de livraison du produit (telephone ou Tablette)
def get_delivery_to(soup):
    return soup.find('div',{'class':'markup -fs12 -pbm'}).get_text()


#La fonction get_link_picture qui retourne le lien de l'image du produit (telephone ou Tablette)
def get_link_picture(soup):
    s=soup.find('div',{'id':'imgs'})
    s=s.find('a')
    return s['href']


#La fonction get_product_data qui retourne tous les caracteristique du produit sous Forme de dictionnaire (telephone ou Tablette)
def get_product_data(product_url):
    r=requests.get(product_url)
#verifions le status code si c'est egale a 200
    if r.status_code==200:  
           #print(r.content)
        soup=BeautifulSoup(r.content, 'html.parser')
#On fait appel aux autres fonctions contenant les informations
        title=get_product_title(soup)
        price=get_product_price(soup).split()
        marque=get_product_mark(soup)[1].get_text().split()
        prix_de_delivrance=get_price_delivery(soup).split()
        note=get_numbers_star(soup).split()
        delivrer=get_delivery_to(soup).split()
        picture=get_link_picture(soup)
#On retourne une dictionnaire contenant tout les informations.
    return { 'Title and feature': title,
            'Price (fcfa)':''.join(price[:-1]),
            'Marque': marque[1],
            'Delivery price': ' '.join(prix_de_delivrance[5:7]),
            'Rating':''.join(note[0]+'/'+note[3]),
            'Delivery to':delivrer[-1],
            'Picture':picture           
           }


#La fonction get_link qui retourne tout les liens concernant les pages a scraper.
def get_link(soup):
    links=[]
    all_link_tags=soup.findAll('article',{'class':'prd _fb col c-prd'})
    for td in all_link_tags:
        a=td.find('a')
        link=a['href']
        links.append('https://www.jumia.sn'+link)
#On retourne une liste contenant tous les liens.
    return links


#Définissons une fonction générique write_csv qui prend une liste de dictionnaires et l'écrit dans 
#un fichier au format CSV. Nous inclurons également les en-têtes de colonne dans la première ligne.
def write_csv(items, path):
    # Ouvrir le fichier en mode écriture
    with open(path, 'w') as f:
        # Retourner s'il n'y a rien à écrire
        if len(items) == 0:
            return
        
        # Écrivez les en-têtes sur la première ligne
        headers = list(items[0].keys())
        f.write(','.join(headers) + '\n')
        
        # Écrire un article par ligne
        for item in items:
            values = []
            for header in headers:
                values.append(str(item.get(header, "")))
            f.write(','.join(values) + "\n")
            


Le Scraping se fait en 2 etapes:                       
**1er etape**: On doit scraper les 14 pages (https://www.jumia.sn/smartphones-android/?sort=popularity&page=({1,2...12})#catalog-listing pour reccuperer les liens de chaques appareils.


In [2]:
t=[]
link=[]

for i in range(1,14):
    r=requests.get('https://www.jumia.sn/smartphones-android/?sort=popularity&page='+str(i)+'#catalog-listing')
    soup=BeautifulSoup(r.content, 'html.parser')
    t.append(get_link(soup))

for i in range(0,len(t)-1):
    for j in range(0,len(t[i])-1):
        link.append(t[i][j])


In [3]:
link

['https://www.jumia.sn/samsung-galaxy-a12-4g-6.5-ram-4go-rom-64go-camera-48522mp-5000mah-bleu-6100197.html',
 'https://www.jumia.sn/samsung-galaxy-a12-4g-6.5-ram-4go-rom-64go-camera-48522mp-5000mah-black-6100196.html',
 'https://www.jumia.sn/tecno-camon-17-ecran-6.55-rom-128go-ram-4go-camera-arriere-48mp-front-8-mp-5000mah-silver-8817329.html',
 'https://www.jumia.sn/tecno-spark-7-6.5-rom-64go-ram-3go-camera-16mp-8mp-5000mah-noir-8850100.html',
 'https://www.jumia.sn/tecno-camon-17-6.6-rom-128go-ram-6go-camera-arriere-48mp-5000mah-argente-8817328.html',
 'https://www.jumia.sn/tecno-pop-2f-5.5-double-sim-rom-16-go-ram-1-go-camera-85-mp-batterie-2400mah-gold-5977479.html',
 'https://www.jumia.sn/tecno-spark-go-6.52-rom-32gb-ram-2gb-camera-138mp-ice-jadette-6044150.html',
 'https://www.jumia.sn/tecno-camon-17-6.55-rom-128go-ram-4go-triple-camera-arriere-48mp-front-8mp-5000mah-deep-sea-8817330.html',
 'https://www.jumia.sn/tecno-pouvoir-4-4g-lte-2-sim-813-mpx-332-go-6000-mah-ice-jadette-59

**2em etape**: Apres avoir Obtenue les liens de chaques appareils nous allons proceder a la recuperation d'informations grace aux fonctions ladite cree.
Nous allons faire appel a la fonction ***get_product_data()***.


In [4]:
#prenons 100 liens et testons notre code.
informations=[]
for i in range(0,100):
    informations.append(get_product_data(link[i]))
    informations[i]['Link']=link[i]

In [5]:
informations

[{'Title and feature': 'Samsung Galaxy A12 - 4G - 6.5" - RAM 4Go - ROM 64Go - Caméra 48+5+2+2MP - 5000mAh – Bleu',
  'Price (fcfa)': '87000',
  'Marque': 'Samsung',
  'Delivery price': '550 FCFA',
  'Rating': '4.4/5',
  'Delivery to': 'Medina',
  'Picture': 'https://sn.jumia.is/unsafe/fit-in/680x680/filters:fill(white)/product/79/10016/1.jpg?6221',
  'Link': 'https://www.jumia.sn/samsung-galaxy-a12-4g-6.5-ram-4go-rom-64go-camera-48522mp-5000mah-bleu-6100197.html'},
 {'Title and feature': 'Samsung Galaxy A12- 4G - 6.5" - RAM 4Go - ROM 64Go - Caméra 48+5+2+2MP - 5000mAh – Black',
  'Price (fcfa)': '87000',
  'Marque': 'Samsung',
  'Delivery price': '550 FCFA',
  'Rating': '4.2/5',
  'Delivery to': 'Medina',
  'Picture': 'https://sn.jumia.is/unsafe/fit-in/680x680/filters:fill(white)/product/69/10016/1.jpg?6221',
  'Link': 'https://www.jumia.sn/samsung-galaxy-a12-4g-6.5-ram-4go-rom-64go-camera-48522mp-5000mah-black-6100196.html'},
 {'Title and feature': 'Tecno Camon 17 - Ecran  6.55" - ROM

In [6]:
#write_csv qui permet de creer un fichier csv a partir d'une liste de dictionnaire
write_csv(informations,'info_csv.txt')

In [7]:
#ouvrons le contenue du fichier 'info_csv.txt'
with open('info_csv.txt', 'r') as f:
    print(f.read())

Title and feature,Price (fcfa),Marque,Delivery price,Rating,Delivery to,Picture,Link
Samsung Galaxy A12 - 4G - 6.5" - RAM 4Go - ROM 64Go - Caméra 48+5+2+2MP - 5000mAh – Bleu,87000,Samsung,550 FCFA,4.4/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filters:fill(white)/product/79/10016/1.jpg?6221,https://www.jumia.sn/samsung-galaxy-a12-4g-6.5-ram-4go-rom-64go-camera-48522mp-5000mah-bleu-6100197.html
Samsung Galaxy A12- 4G - 6.5" - RAM 4Go - ROM 64Go - Caméra 48+5+2+2MP - 5000mAh – Black,87000,Samsung,550 FCFA,4.2/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filters:fill(white)/product/69/10016/1.jpg?6221,https://www.jumia.sn/samsung-galaxy-a12-4g-6.5-ram-4go-rom-64go-camera-48522mp-5000mah-black-6100196.html
Tecno Camon 17 - Ecran  6.55" - ROM 128Go - RAM 4Go - Camera arrière 48MP - front 8 MP-  5000mAh - Silver,93000,Tecno,550 FCFA,4.1/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filters:fill(white)/product/92/37188/1.jpg?6184,https://www.jumia.sn/tecno-camon-17-ecran-6.

In [8]:
import pandas as pd

In [9]:
#creons un dataframe a partir du fichier csv 'info_csv.txt' en eliminant toute les lignes comportant des erreurs .
data=pd.read_csv('info_csv.txt',sep=',',error_bad_lines=None,warn_bad_lines=False)
pd.set_option('mode.chained_assignment', None)
#r:le nombre de ligne du dataframe
r=data.shape[0]


In [10]:
#Parcourons la colonne 'price' de notre dataframe en corrigeant certaine erreurs
for i in range(0,r-1):
    try:
        data['Price (fcfa)'][i]=int(data['Price (fcfa)'][i])
    except ValueError as e:
        s=data['Price (fcfa)'][i].split('-')
        data['Price (fcfa)'][i]=s[1]
        data['Price (fcfa)'][i]=float(data['Price (fcfa)'][i])
        
#Convertir la colonne Price en float
data['Price (fcfa)']=data['Price (fcfa)'].astype(int)

In [11]:
#Pour afficher tous les colonnes de notre dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [12]:
data

,Title and feature,Price (fcfa),Marque,Delivery price,Rating,Delivery to,Picture,Link
0,"Samsung Galaxy A12 - 4G - 6.5"" - RAM 4Go - ROM...",87000,Samsung,550 FCFA,4.4/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/samsung-galaxy-a12-4g-6.5...
1,"Samsung Galaxy A12- 4G - 6.5"" - RAM 4Go - ROM ...",87000,Samsung,550 FCFA,4.2/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/samsung-galaxy-a12-4g-6.5...
2,"Tecno Camon 17 - Ecran 6.55"" - ROM 128Go - RA...",93000,Tecno,550 FCFA,4.1/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/tecno-camon-17-ecran-6.55...
3,"Tecno Spark 7 - 6.5"" - ROM 64Go - RAM 3Go - Ca...",69600,Tecno,550 FCFA,4.2/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/tecno-spark-7-6.5-rom-64g...
4,"Tecno Camon 17 - 6.6"" - ROM 128Go - RAM 6Go - ...",101000,Tecno,550 FCFA,4.2/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/tecno-camon-17-6.6-rom-12...
5,Tecno POP 2F - 5.5’’ - Double SIM - ROM 16 Go ...,35500,Tecno,550 FCFA,4.8/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/tecno-pop-2f-5.5-double-s...
6,"Tecno SPARK Go - 6.52"" - ROM 32GB - RAM 2GB -...",51900,Tecno,550 FCFA,4.3/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/tecno-spark-go-6.52-rom-3...
7,"Tecno Camon 17 - 6.55"" - ROM 128Go - RAM 4Go -...",92300,Tecno,550 FCFA,4.2/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/tecno-camon-17-6.55-rom-1...
8,Tecno Pouvoir 4 - 4G LTE - 2 SIM - 8/13 Mpx - ...,62900,Tecno,550 FCFA,4.7/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/tecno-pouvoir-4-4g-lte-2-...
9,"Tecno Camon 15 - 6.6"" - ROM 64Go - RAM 4Go - C...",74900,Tecno,550 FCFA,4.2/5,Medina,https://sn.jumia.is/unsafe/fit-in/680x680/filt...,https://www.jumia.sn/tecno-camon-15-6.6-rom-64...


In [15]:
#Renvoie l'indice Du Maximum dans 'Price (fcfa)'
data['Price (fcfa)'].argmax()

79

In [16]:
#Renvoie la ligne de l'appareil la plus chere
s=data.loc[data['Price (fcfa)'].argmax()]
s

Title and feature    Redmi Note 10 Pro Max – Ecran 6.67″ - ROM 128G...
Price (fcfa)                                                    178000
Marque                                                           Redmi
Delivery price                                                550 FCFA
Rating                                                             0/5
Delivery to                                                     Medina
Picture              https://sn.jumia.is/unsafe/fit-in/680x680/filt...
Link                 https://www.jumia.sn/redmi-note-10-pro-max-ecr...
Name: 79, dtype: object

In [17]:
#Pour afficher l'image
print(s['Picture'])

https://sn.jumia.is/unsafe/fit-in/680x680/filters:fill(white)/product/64/09429/1.jpg?8064


In [18]:
#Pour voir le lien de l'appareil la plus chere
print(s['Link'])

https://www.jumia.sn/redmi-note-10-pro-max-ecran-6.67-rom-128gb-ram-6gb-camera-108mp-9249046.html
